In [5]:
import boto3
import pandas as pd
import pymysql
import psycopg2
from datetime import datetime
from dateutil.relativedelta import relativedelta
import numpy as np
import warnings
warnings.filterwarnings("ignore")
# pd.set_option('display.max_columns', None)

In [4]:
#!pip install s3fs

In [5]:
def get_raw_data():
#     final_df= pd.read_csv('isw_data.csv')
    final_df= pd.read_csv('isw_all_data.csv')
    final_df.reset_index(dropTt=True, inplace=True)
    return final_df
final_df=get_raw_data()
final_df

,Unnamed: 0,time_,credit_amt,debit_amt,terminal,biller,narration,balance,trxn_type,status,request_ref
0,0,2023-05-03 19:57,"50,000",0.00,3IS00014,ISW Paypoint Agent,airtimeanddatafacilitaionformarch,"158,263",Payment,Approved,CR-000000245220
1,1,2023-05-04 21:06,"300,000",0.00,3IS00014,ISW Paypoint Agent,KYC,"458,263",Payment,Approved,CR-000000245526
2,2,2023-05-05 07:59,0.00,"300,000",3IS00014,Airtel Deposit,NaN,"158,263",Payment,Approved,ISWM16832627651440014
3,3,2023-05-06 14:11,0.00,200,3IS00014,Airtel Airtime,NaN,"158,063",Payment,Approved,ISWM16833715061870014
4,4,2023-05-09 01:47,"9,536",0.00,3IS00014,Agent Commission,CommissionApril2023,"167,599",Payment,Approved,CR-000000249736
...,...,...,...,...,...,...,...,...,...,...,...
4140527,4140527,2023-07-13 15:11,0.00,"1,000",3IS03965,Airtel Airtime,NaN,"1,754,486",Payment,Approved,ISWM16892503175853965
4140528,4140528,2023-07-13 15:17,0.00,"2,000",3IS03965,Airtel Airtime,NaN,"1,752,486",Payment,Approved,ISWM16892506426303965
4140529,4140529,2023-07-13 15:54,0.00,"1,000",3IS03965,MTN DEPOSIT,NaN,"1,751,486",Payment,Approved,ISWM16892528619763965
4140530,4140530,2023-07-13 16:36,0.00,"1,000",3IS03965,MTN DEPOSIT,NaN,"1,750,486",Payment,Approved,ISWM16892554263353965


In [6]:
# terminal =  [terminal for terminal in final_df["terminal"]]
# len(terminal)
# final_df["terminal"].nunique()


In [7]:
#  terminal_ids: ["3IS16325", "3IS12910", "3IS34786",
#                 "3IS00720", "3IS08111"]

# terminal_id = terminal
# '3IS00564'
# terminal =  [terminal for terminal in final_df["terminal"].nunique()] 


# final_df=final_df[final_df["terminal"]==terminal]
# final_df
# bras=[]

# for terminal in final_df["terminal"].unique():
#     final_df = final_df[final_df["terminal"] == terminal]
#     bras.append(final_df)
# bras

In [8]:
def get_raw_data():
    s3_client = boto3.client('s3')
    bucket_name = 'afsg-dgt-prod-iswug-file-upload'
    
    objects = s3_client.list_objects(Bucket=bucket_name)
    csv_files = [obj['Key'] for obj in objects.get('Contents', []) if obj['Key'].endswith('.csv')]
    
    dataframes = []

    for csv_file in csv_files:
        s3_object = s3_client.get_object(Bucket=bucket_name, Key=csv_file)
        df = pd.read_csv(s3_object['Body'])
        
        if 'Terminal' in df.columns and df['Terminal'].iloc[0] == terminal:
            dataframes.append(df)
    combined_dataframe = pd.concat(dataframes, ignore_index=True)
    combined_dataframe.rename(columns={
                'Date': 'time_', 'Terminal': 'terminal', 'RequestRef': 'request_ref',
                'TranDesc': 'trxn_type', 'Biller': 'biller', 'Narration': 'narration',
                'DebitAmount': 'debit_amt', 'CreditAmount': 'credit_amt', 'Balance': 'balance', 'Status': 'status'
            }, inplace=True)

    return combined_dataframe

In [9]:

def get_loans_data():


    MIFOS_HOST= "167.71.47.216"
    MIFOS_PORT=3306
    MIFOS_DB_NAME= "mifostenant-pronto"
    MIFOS_USER="soguna"
    MIFOS_PASSWORD= "Change@123"


    connection = pymysql.connect(
        host=MIFOS_HOST,
        user=MIFOS_USER,
        password=MIFOS_PASSWORD,
        database=MIFOS_DB_NAME,
        port=MIFOS_PORT
    )


#     query = f"""
#             SELECT 
#                 id, client_id, principal_disbursed_derived, disbursedon_date,closedon_date, expected_maturedon_date,
#                 total_expected_repayment_derived
#             FROM `mifostenant-uganda`.m_loan 
#             where product_id IN ( 2, 3) and disbursedon_date is not null 
#             """
    query = f"""
             SELECT
    mc.external_id,ml.product_id,ml.disbursedon_date,ml.expected_maturedon_date,ml.maturedon_date,ml.closedon_date
    
FROM
    `mifostenant-uganda`.m_loan ml
LEFT JOIN
    `mifostenant-uganda`.m_client mc ON ml.client_id = mc.id
WHERE
    ml.product_id IN (2,3)
    AND ml.disbursedon_date IS NOT NULL and ml.expected_maturedon_date IS NOT NULL and ml.closedon_date IS NOT NULL
    
    """
            
     # --- and loan_status_id in (300,600,700) and client_id = '{terminal_id}'
    df = pd.read_sql(query, connection)
    
#     condition = df["external_id"].value_counts() > 1
#     repeat_customers = df.loc[loans["external_id"].isin(condition[condition].index)]
#     repeat_customers


    connection.close()
    
    return df

loans = get_loans_data()
loans.shape

loans['disbursedon_date'] = pd.to_datetime(loans['disbursedon_date'], errors='coerce')
loans['expected_maturedon_date']=pd.to_datetime(loans['expected_maturedon_date'], errors='coerce')
loans['closedon_date']=pd.to_datetime(loans['closedon_date'])
condition = loans["external_id"].value_counts() > 1
repeat_customers = loans.loc[loans["external_id"].isin(condition[condition].index)]
repeat_customers2=repeat_customers.copy()
repeat_customers2.shape

def add(df):        
    if (df["expected_maturedon_date"] >= df["closedon_date"]):
        return False
    else:
        return True
    
repeat_customers2['Dpd'] = repeat_customers2.apply(lambda x: add(x), axis=1)
repeat_customers2.to_csv("ISW_refined_loans_data.csv")
repeat_customers2



,external_id,product_id,disbursedon_date,expected_maturedon_date,maturedon_date,closedon_date,Dpd
0,3IS34537,2,2022-11-08,2022-11-15,2022-11-08,2022-11-08,False
1,3IS34537,2,2022-11-08,2022-11-15,2022-11-09,2022-11-09,False
2,3IS34537,2,2022-11-09,2022-11-16,2022-11-09,2022-11-09,False
3,3IS34537,2,2022-11-09,2022-11-16,2022-11-11,2022-11-11,False
4,3IS34537,2,2022-11-11,2022-11-18,2022-11-11,2022-11-11,False
...,...,...,...,...,...,...,...
4118,3IS06404,3,2023-11-23,2023-11-26,2023-11-27,2023-11-27,True
4119,3IS03330,3,2023-11-24,2023-11-27,2023-11-27,2023-11-27,False
4120,3IS34617,3,2023-11-24,2023-11-27,2023-11-27,2023-11-27,False
4121,3IS16778,2,2023-11-24,2023-12-01,2023-11-24,2023-11-24,False


In [10]:
# def get_transaction_data():


#     MIFOS_HOST= "167.71.47.216"
#     MIFOS_PORT=3306
#     MIFOS_DB_NAME= "mifostenant-pronto"
#     MIFOS_USER="soguna"
#     MIFOS_PASSWORD= "Change@123"


#     connection = pymysql.connect(
#         host=MIFOS_HOST,
#         user=MIFOS_USER,
#         password=MIFOS_PASSWORD,
#         database=MIFOS_DB_NAME,
#         port=MIFOS_PORT
#     )


#     query = f"""
#         SELECT 
#                 mlt.loan_id, mlt.transaction_date, mlt.amount 
#             from `mifostenant-uganda`.m_loan ml left join
#             `mifostenant-uganda`.m_loan_transaction mlt on ml.id = mlt.loan_id  
#             where mlt.transaction_type_enum = 2 and mlt.is_reversed = 0
#             and ml.client_id = '{terminal_id}'
#         """

#     df = pd.read_sql(query, connection)


#     connection.close()
    
#     return df



In [11]:
# transactions = get_transaction_data()
# transactions 

In [12]:
# terminal_id = terminal

In [13]:

# def get_prev_month_scoring_result():
    
#     DWH_HOST="afsg-ds-prod-postgresql-dwh-identifier.ct59jfgaa51w.eu-central-1.rds.amazonaws.com"
#     DWH_PORT="5432"
#     DWH_DB_NAME="afsg_ds_prod_postgresql_dwh"
#     DWH_USER= "stephenoguna"
#     DWH_PASSWORD= "bZD71J^p4LJhKroF"
#     db_params = {
#         'host': DWH_HOST,
#         'port':DWH_PORT,
#         'database': DWH_DB_NAME,
#         'user': DWH_USER,
#         'password': DWH_PASSWORD,
#     }
#     snapshot_weeks=8

#     conn = psycopg2.connect(**db_params)

    
#     cur = conn.cursor()

#     query = f"""
#         WITH sr_prev_m AS (
#                             SELECT
#                                 sr_prev_m.terminal 
#                                 ,MAX(sr_prev_m.model_version) AS previous_m_model_version
#                                 ,MAX(sr_prev_m.id) AS id
#                             FROM interswitch_ug.scoring_results sr_prev_m 
#                             --WHERE DATE_PART('week', SUBSTRING(sr_prev_m.model_version,'\s.*\d')::DATE) < DATE_PART('week', NOW()::DATE - '{snapshot_weeks}weeks'::INTERVAL)
#                             WHERE SUBSTRING(sr_prev_m.model_version,'\s.*\d')::DATE < NOW()::DATE - '1month'::INTERVAL
#                                 AND sr_prev_m.terminal = '{terminal_id}'
#                             GROUP BY 
#                                 sr_prev_m.terminal
#                             ),
#             sr_prev_2m AS (
#                             SELECT
#                                 sr_prev_m.terminal 
#                                 ,MAX(sr_prev_m.model_version) AS previous_2m_model_version
#                                 ,MAX(sr_prev_m.id) AS id
#                             FROM interswitch_ug.scoring_results sr_prev_m 
#                             --WHERE DATE_PART('week', SUBSTRING(sr_prev_m.model_version,'\s.*\d')::DATE) < DATE_PART('week', NOW()::DATE - '{snapshot_weeks}weeks'::INTERVAL)
#                             WHERE SUBSTRING(sr_prev_m.model_version,'\s.*\d')::DATE < NOW()::DATE - '2months'::INTERVAL
#                                 AND sr_prev_m.terminal = '{terminal_id}'
#                             GROUP BY 
#                                 sr_prev_m.terminal
#                             ),
#             limit_prev_m AS (
#                             SELECT 
#                                 sr_prev_m.terminal 
#                                 --,sr_prev_m.id
#                                 --,sr_prev_m.previous_m_model_version
#                                 ,sr.final_3_day_limit AS previous_m_limit
#                             FROM sr_prev_m
#                             LEFT JOIN interswitch_ug.scoring_results sr
#                                 ON sr_prev_m.terminal = sr.terminal
#                                 AND sr_prev_m.previous_m_model_version = sr.model_version
#                                 AND sr_prev_m.id = sr.id
#                             WHERE sr.final_3_day_limit IS NOT NULL
#                                 AND sr_prev_m.terminal = '{terminal_id}'
#                             /*ORDER BY 
#                                 sr.model_version DESC*/
#                             ),
#             limit_prev_2m AS (
#                             SELECT 
#                                 sr_prev_2m.terminal 
#                                 --,sr_prev_2m.id
#                                 --,sr_prev_2m.previous_m_model_version
#                                 ,sr.final_3_day_limit AS previous_2m_limit
#                             FROM sr_prev_2m
#                             LEFT JOIN interswitch_ug.scoring_results sr
#                                 ON sr_prev_2m.terminal = sr.terminal
#                                 AND sr_prev_2m.previous_2m_model_version = sr.model_version
#                                 AND sr_prev_2m.id = sr.id
#                             WHERE sr.final_3_day_limit IS NOT NULL
#                                 AND sr_prev_2m.terminal = '{terminal_id}'
#                             /*ORDER BY 
#                                 sr.model_version DESC*/
#                             )
#         SELECT 
#             limit_prev_m.terminal 
#             ,limit_prev_m.previous_m_limit
#             ,limit_prev_2m.previous_2m_limit
#         FROM limit_prev_m
#         LEFT JOIN  limit_prev_2m
#             ON limit_prev_m.terminal = limit_prev_2m.terminal
#         """

#     cur.execute(query)

    
#     results = cur.fetchall()
#     column_names=['terminal','previous_m_limit','previous_2m_limit']

   
#     df = pd.DataFrame(results,columns=column_names)

    
    

    
#     cur.close()
#     conn.close()
#     return df

In [14]:
# prev_month_results = get_prev_month_scoring_result()
# prev_month_results 

In [15]:
# def load_staging_db_data():


#     ISW_HOST = "10.0.1.222"
#     ISW_PORT = 3306
#     ISW_DB_NAME = "iswug_staging"
#     ISW_USER = "stephen01"
#     ISW_PASSWORD = "Stephen01@2023"


#     connection = pymysql.connect(
#         host=ISW_HOST,
#         user=ISW_USER,
#         password=ISW_PASSWORD,
#         database=ISW_DB_NAME,
#         port=ISW_PORT
#     )

#     terminal_id ='3IS08058'
#     query = f"""
#         SELECT TerminalId as terminal, MifosClientId as client_id FROM Customers where TerminalId = '{terminal_id}'
#         """

#     df = pd.read_sql(query, connection)


#     connection.close()
    
#     return df



In [16]:
# client_data = load_staging_db_data()
# client_data

In [17]:
# def get_prev_scoring_results():
#     DWH_HOST="afsg-ds-prod-postgresql-dwh-identifier.ct59jfgaa51w.eu-central-1.rds.amazonaws.com"
#     DWH_PORT="5432"
#     DWH_DB_NAME="afsg_ds_prod_postgresql_dwh"
#     DWH_USER= "stephenoguna"
#     DWH_PASSWORD= "bZD71J^p4LJhKroF"
#     db_params = {
#         'host': DWH_HOST,
#         'port':DWH_PORT,
#         'database': DWH_DB_NAME,
#         'user': DWH_USER,
#         'password': DWH_PASSWORD,
#     }
   
#     conn = psycopg2.connect(**db_params)

    
#     cur = conn.cursor()

#     query =  f"""
#         SELECT terminal, final_3_day_limit as previous_limit FROM interswitch_ug.scoring_results_view where terminal = '{terminal_id}'
#         """
        
                            

#     cur.execute(query)
#     column_names = ['terminal','previous_limit']
    
#     results = cur.fetchall()

   
#     df = pd.DataFrame(results,columns=column_names)

    
    

    
#     cur.close()
#     conn.close()
#     return df

In [18]:
# previous_results=get_prev_scoring_results()
# previous_results

# View Data_Sets

In [19]:
def process_dates():
    df =final_df
#     df['time_'] = pd.to_datetime(df['time_'], format='%d/%m/%Y %H:%M')
    df['time_'] = pd.to_datetime(df['time_'], format='%Y-%m-%d %H:%M', errors='coerce', infer_datetime_format=True)


    df['transaction_dates'] = [x.strftime('%d-%m-%Y') for x in df['time_']]

    df['transaction_dates'] = pd.to_datetime(df['transaction_dates'], format='%d-%m-%Y')

    df['year_month_transaction_dates'] = [x.strftime('%b-%Y') for x in df['time_']]

    return df

In [20]:
raw_data=process_dates()
raw_data
#raw_data= raw_data[raw_data['terminal'] == terminal_id]

,Unnamed: 0,time_,credit_amt,debit_amt,terminal,biller,narration,balance,trxn_type,status,request_ref,transaction_dates,year_month_transaction_dates
0,0,2023-05-03 19:57:00,"50,000",0.00,3IS00014,ISW Paypoint Agent,airtimeanddatafacilitaionformarch,"158,263",Payment,Approved,CR-000000245220,2023-05-03,May-2023
1,1,2023-05-04 21:06:00,"300,000",0.00,3IS00014,ISW Paypoint Agent,KYC,"458,263",Payment,Approved,CR-000000245526,2023-05-04,May-2023
2,2,2023-05-05 07:59:00,0.00,"300,000",3IS00014,Airtel Deposit,NaN,"158,263",Payment,Approved,ISWM16832627651440014,2023-05-05,May-2023
3,3,2023-05-06 14:11:00,0.00,200,3IS00014,Airtel Airtime,NaN,"158,063",Payment,Approved,ISWM16833715061870014,2023-05-06,May-2023
4,4,2023-05-09 01:47:00,"9,536",0.00,3IS00014,Agent Commission,CommissionApril2023,"167,599",Payment,Approved,CR-000000249736,2023-05-09,May-2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4140527,4140527,2023-07-13 15:11:00,0.00,"1,000",3IS03965,Airtel Airtime,NaN,"1,754,486",Payment,Approved,ISWM16892503175853965,2023-07-13,Jul-2023
4140528,4140528,2023-07-13 15:17:00,0.00,"2,000",3IS03965,Airtel Airtime,NaN,"1,752,486",Payment,Approved,ISWM16892506426303965,2023-07-13,Jul-2023
4140529,4140529,2023-07-13 15:54:00,0.00,"1,000",3IS03965,MTN DEPOSIT,NaN,"1,751,486",Payment,Approved,ISWM16892528619763965,2023-07-13,Jul-2023
4140530,4140530,2023-07-13 16:36:00,0.00,"1,000",3IS03965,MTN DEPOSIT,NaN,"1,750,486",Payment,Approved,ISWM16892554263353965,2023-07-13,Jul-2023


In [21]:
# column_index_to_drop = 0
# raw_data.drop(raw_data.columns[column_index_to_drop], axis=1, inplace=True)
# raw_data
# raw_data
raw_data.drop(raw_data.columns[raw_data.columns.str.contains('Unnamed')].tolist(), axis=1, inplace=True)
raw_data

,time_,credit_amt,debit_amt,terminal,biller,narration,balance,trxn_type,status,request_ref,transaction_dates,year_month_transaction_dates
0,2023-05-03 19:57:00,"50,000",0.00,3IS00014,ISW Paypoint Agent,airtimeanddatafacilitaionformarch,"158,263",Payment,Approved,CR-000000245220,2023-05-03,May-2023
1,2023-05-04 21:06:00,"300,000",0.00,3IS00014,ISW Paypoint Agent,KYC,"458,263",Payment,Approved,CR-000000245526,2023-05-04,May-2023
2,2023-05-05 07:59:00,0.00,"300,000",3IS00014,Airtel Deposit,NaN,"158,263",Payment,Approved,ISWM16832627651440014,2023-05-05,May-2023
3,2023-05-06 14:11:00,0.00,200,3IS00014,Airtel Airtime,NaN,"158,063",Payment,Approved,ISWM16833715061870014,2023-05-06,May-2023
4,2023-05-09 01:47:00,"9,536",0.00,3IS00014,Agent Commission,CommissionApril2023,"167,599",Payment,Approved,CR-000000249736,2023-05-09,May-2023
...,...,...,...,...,...,...,...,...,...,...,...,...
4140527,2023-07-13 15:11:00,0.00,"1,000",3IS03965,Airtel Airtime,NaN,"1,754,486",Payment,Approved,ISWM16892503175853965,2023-07-13,Jul-2023
4140528,2023-07-13 15:17:00,0.00,"2,000",3IS03965,Airtel Airtime,NaN,"1,752,486",Payment,Approved,ISWM16892506426303965,2023-07-13,Jul-2023
4140529,2023-07-13 15:54:00,0.00,"1,000",3IS03965,MTN DEPOSIT,NaN,"1,751,486",Payment,Approved,ISWM16892528619763965,2023-07-13,Jul-2023
4140530,2023-07-13 16:36:00,0.00,"1,000",3IS03965,MTN DEPOSIT,NaN,"1,750,486",Payment,Approved,ISWM16892554263353965,2023-07-13,Jul-2023


### Commisison Summaries

In [22]:
#Last 6 months comission summaries
def calculate_commissions_summaries_last_6_months(data):
    raw_data = process_dates()
    df= process_dates()
    max_transactions_date = pd.to_datetime('today')

    # max_transactions_date = df['transaction_dates'].max()
    

    df_last_6_months = df[df['transaction_dates'] > max_transactions_date + relativedelta(months=-6)]
    df_last_6_months['credit_amt'] = pd.to_numeric(df_last_6_months['credit_amt'], errors='coerce')

    commissions = df_last_6_months[
        (df_last_6_months['biller'] == 'Agent Commission') & (df_last_6_months['status'] == 'Approved')]

    commissions_summaries = commissions.groupby('terminal').agg(
        total_commissions_amount_last_6_month=pd.NamedAgg('credit_amt', aggfunc='sum'),
        unique_number_of_commissions_last_6_month=pd.NamedAgg('transaction_dates', aggfunc='nunique'),
        unique_number_of_services_offered_last_6_month=pd.NamedAgg('narration', aggfunc='nunique'),
        number_of_months_received_commissions_last_6_month=pd.NamedAgg('year_month_transaction_dates',
                                                          aggfunc='nunique')).reset_index()
    commissions_summaries['average_commisison_last_6_months']=commissions_summaries['total_commissions_amount_last_6_month']/commissions_summaries['unique_number_of_commissions_last_6_month']
    
    
    commissions_summaries["total_commissions_amount_last_6_month"].fillna(0, inplace=True)
    commissions_summaries["unique_number_of_commissions_last_6_month"].fillna(0, inplace=True)
    commissions_summaries["unique_number_of_services_offered_last_6_month"].fillna(0, inplace=True)
    commissions_summaries["number_of_months_received_commissions_last_6_month"].fillna(0, inplace=True)
    commissions_summaries["average_commisison_last_6_months"].fillna(0, inplace=True)


    return commissions_summaries


In [23]:
# last_6_months_commissions_summaries=calculate_commissions_summaries_last_6_months()
# last_6_months_commissions_summaries

In [24]:
# nan_counts = last_6_months_commissions_summaries.isna().sum()
# nan_counts

In [25]:
#Last 6 months comission summaries
def calculate_commissions_summaries_last_1_months(data):
#     raw_data = process_dates()
#     df= process_dates()
    df=data
    max_transactions_date = pd.to_datetime('today')

    # max_transactions_date = df['transaction_dates'].max()
    

    df_last_1_months = df[df['transaction_dates'] > max_transactions_date + relativedelta(months=-1)]
    df_last_1_months['credit_amt'] = pd.to_numeric(df_last_1_months['credit_amt'], errors='coerce')

    commissions = df_last_1_months[
        (df_last_1_months['biller'] == 'Agent Commission') & (df_last_1_months['status'] == 'Approved')]

    commissions_summaries = commissions.groupby('terminal').agg(
        total_commissions_amount_last_1_month=pd.NamedAgg('credit_amt', aggfunc='sum'),
        unique_number_of_commissions_last_1_month=pd.NamedAgg('transaction_dates', aggfunc='nunique'),
        unique_number_of_services_offered_last_1_month=pd.NamedAgg('narration', aggfunc='nunique'),
        number_of_months_received_commissions_last_1_month=pd.NamedAgg('year_month_transaction_dates',
                                                          aggfunc='nunique')).reset_index()
    commissions_summaries['average_commisison_last_1_months']=commissions_summaries['total_commissions_amount_last_1_month']/commissions_summaries['unique_number_of_commissions_last_1_month']
    
    commissions_summaries["total_commissions_amount_last_1_month"].fillna(0, inplace=True)
    commissions_summaries["unique_number_of_commissions_last_1_month"].fillna(0, inplace=True)
    commissions_summaries["unique_number_of_services_offered_last_1_month"].fillna(0, inplace=True)
    commissions_summaries["number_of_months_received_commissions_last_1_month"].fillna(0, inplace=True)
    commissions_summaries["average_commisison_last_1_months"].fillna(0, inplace=True)
    

    return commissions_summaries


In [26]:
# last_1_months_commissions_summaries=calculate_commissions_summaries_last_1_months()
# last_1_months_commissions_summaries

In [27]:
# nan_counts_1 = last_1_months_commissions_summaries.isnull().sum()
# nan_counts_1

In [28]:
#Last 6 months comission summaries
def calculate_commissions_summaries_last_3_months(data):
#     raw_data = process_dates()
#     df= process_dates()
    df=data
    max_transactions_date = pd.to_datetime('today')

    # max_transactions_date = df['transaction_dates'].max()
    

    df_last_3_months = df[df['transaction_dates'] > max_transactions_date + relativedelta(months=-3)]
    df_last_3_months['credit_amt'] = pd.to_numeric(df_last_3_months['credit_amt'], errors='coerce')

    commissions = df_last_3_months[
        (df_last_3_months['biller'] == 'Agent Commission') & (df_last_3_months['status'] == 'Approved')]

    commissions_summaries = commissions.groupby('terminal').agg(
        total_commissions_amount_last_3_month=pd.NamedAgg('credit_amt', aggfunc='sum'),
        unique_number_of_commissions_last_3_month=pd.NamedAgg('transaction_dates', aggfunc='nunique'),
        unique_number_of_services_offered_last_3_month=pd.NamedAgg('narration', aggfunc='nunique'),
        number_of_months_received_commissions_last_3_month=pd.NamedAgg('year_month_transaction_dates',
                                                          aggfunc='nunique')).reset_index()
    commissions_summaries['average_commisison_last_3_months']=commissions_summaries['total_commissions_amount_last_3_month']/commissions_summaries['unique_number_of_commissions_last_3_month']
    
    commissions_summaries["total_commissions_amount_last_3_month"].fillna(0, inplace=True)
    commissions_summaries["unique_number_of_commissions_last_3_month"].fillna(0, inplace=True)
    commissions_summaries["unique_number_of_services_offered_last_3_month"].fillna(0, inplace=True)
    commissions_summaries["number_of_months_received_commissions_last_3_month"].fillna(0, inplace=True)
    commissions_summaries["average_commisison_last_3_months"].fillna(0, inplace=True)
    

    return commissions_summaries


In [29]:
# last_3_months_commissions_summaries=calculate_commissions_summaries_last_3_months()
# last_3_months_commissions_summaries

In [30]:
#Last 6 months comission summaries
def calculate_commissions_summaries_last_9_months(data):
#     raw_data = process_dates()
#     df= process_dates()
    df=data
    max_transactions_date = pd.to_datetime('today')

    # max_transactions_date = df['transaction_dates'].max()
    

    df_last_9_months = df[df['transaction_dates'] > max_transactions_date + relativedelta(months=-9)]
    df_last_9_months['credit_amt'] = pd.to_numeric(df_last_9_months['credit_amt'], errors='coerce')

    commissions = df_last_9_months[
        (df_last_9_months['biller'] == 'Agent Commission') & (df_last_9_months['status'] == 'Approved')]

    commissions_summaries = commissions.groupby('terminal').agg(
        total_commissions_amount_last_9_month=pd.NamedAgg('credit_amt', aggfunc='sum'),
        unique_number_of_commissions_last_9_month=pd.NamedAgg('transaction_dates', aggfunc='nunique'),
        unique_number_of_services_offered_last_9_month=pd.NamedAgg('narration', aggfunc='nunique'),
        number_of_months_received_commissions_last_9_month=pd.NamedAgg('year_month_transaction_dates',
                                                          aggfunc='nunique')).reset_index()
    commissions_summaries['average_commisison_last_9_months']=commissions_summaries['total_commissions_amount_last_9_month']/commissions_summaries['unique_number_of_commissions_last_9_month']
    
    commissions_summaries["total_commissions_amount_last_9_month"].fillna(0, inplace=True)
    commissions_summaries["unique_number_of_commissions_last_9_month"].fillna(0, inplace=True)
    commissions_summaries["unique_number_of_services_offered_last_9_month"].fillna(0, inplace=True)
    commissions_summaries["number_of_months_received_commissions_last_9_month"].fillna(0, inplace=True)
    commissions_summaries["average_commisison_last_9_months"].fillna(0, inplace=True)
    

    return commissions_summaries


In [31]:
# last_9_months_commissions_summaries=calculate_commissions_summaries_last_9_months()
# last_9_months_commissions_summaries

In [32]:
#Last 6 months comission summaries
def calculate_commissions_summaries_last_12_months(data):
#     raw_data = process_dates()
#     df= process_dates()
    df=data
    max_transactions_date = pd.to_datetime('today')

    # max_transactions_date = df['transaction_dates'].max()
    

    df_last_12_months = df[df['transaction_dates'] > max_transactions_date + relativedelta(months=-12)]
    df_last_12_months['credit_amt'] = pd.to_numeric(df_last_12_months['credit_amt'], errors='coerce')

    commissions = df_last_12_months[
        (df_last_12_months['biller'] == 'Agent Commission') & (df_last_12_months['status'] == 'Approved')]

    commissions_summaries = commissions.groupby('terminal').agg(
        total_commissions_amount_last_12_month=pd.NamedAgg('credit_amt', aggfunc='sum'),
        unique_number_of_commissions_last_12_month=pd.NamedAgg('transaction_dates', aggfunc='nunique'),
        unique_number_of_services_offered_last_12_month=pd.NamedAgg('narration', aggfunc='nunique'),
        number_of_months_received_commissions_last_12_month=pd.NamedAgg('year_month_transaction_dates',
                                                          aggfunc='nunique')).reset_index()
    commissions_summaries['average_commisison_last_12_months']=commissions_summaries['total_commissions_amount_last_12_month']/commissions_summaries['unique_number_of_commissions_last_12_month']
    
    commissions_summaries["total_commissions_amount_last_12_month"].fillna(0, inplace=True)
    commissions_summaries["unique_number_of_commissions_last_12_month"].fillna(0, inplace=True)
    commissions_summaries["unique_number_of_services_offered_last_12_month"].fillna(0, inplace=True)
    commissions_summaries["number_of_months_received_commissions_last_12_month"].fillna(0, inplace=True)
    commissions_summaries["average_commisison_last_12_months"].fillna(0, inplace=True)
    

    return commissions_summaries


In [33]:
# last_12_months_commissions_summaries=calculate_commissions_summaries_last_12_months()
# last_12_months_commissions_summaries

### Debits Scoring Summaries 

In [34]:
def calculate_6_months_debits_scoring_summaries(data):
#     df=process_dates()
    df=data
    
    max_transactions_date = pd.to_datetime('today')
    #max_transactions_date = df['transaction_dates'].max()

    df_last_6_months = df[df['transaction_dates'] >= max_transactions_date + relativedelta(months=-6)]
    
    columns_to_convert = ['credit_amt', 'debit_amt','balance']

    for column in columns_to_convert:
        df_last_6_months[column] = pd.to_numeric(df_last_6_months[column], errors='coerce')
        df_last_6_months[column].fillna(0,inplace=True)


    debits_df_last_6_months = df_last_6_months[
        (df_last_6_months['debit_amt'] > 0) & (df_last_6_months['status'] == 'Approved')]

    debits_df_last_6_months_negative_balances = debits_df_last_6_months[debits_df_last_6_months['balance'] < 0]

    negative_balances_summaries_last_6_months = debits_df_last_6_months_negative_balances.groupby('terminal').agg(
        debits_lowest_negative_balance_last_6_months=pd.NamedAgg('balance', min),
        debits_highest_negative_balance_last_6_months=pd.NamedAgg('balance', max),
        debits_earliest_negative_balance_date_last_6_months=pd.NamedAgg('transaction_dates', min),
        debits_latest_negative_balance_date_last_6_months=pd.NamedAgg('transaction_dates', max),
        debits_unique_negative_balance_dates_last_6_months=pd.NamedAgg('transaction_dates', 'nunique')).reset_index()
    

    debits_df_last_6_months_without_negative_balances = debits_df_last_6_months[debits_df_last_6_months['balance'] >= 0]

    agent_summaries_last_6_months = debits_df_last_6_months_without_negative_balances.groupby('terminal').agg(
        debits_total_debit_amount_last_6_months=pd.NamedAgg('debit_amt', sum),
        debits_average_transaction_size_last_6_months=pd.NamedAgg('debit_amt', 'mean'),
        debits_total_transactions_last_6_months=pd.NamedAgg('request_ref', 'nunique'),
        debits_earliest_transaction_date_last_6_months=pd.NamedAgg('transaction_dates', min),
        debits_latest_transaction_date_last_6_months=pd.NamedAgg('transaction_dates', max),
        debits_no_of_services_offered_last_6_months=pd.NamedAgg('biller', 'nunique'),
        debits_min_balance_last_6_months=pd.NamedAgg('balance', min),
        debits_max_balance_last_6_months=pd.NamedAgg('balance', max),
        debits_average_balance_last_6_months=pd.NamedAgg('balance', 'mean'),
        debits_min_debit_amt_last_6_months=pd.NamedAgg('debit_amt', min),
        debits_max_debit_amt_last_6_months=pd.NamedAgg('debit_amt', max),
        debits_unique_transaction_days_last_6_months=pd.NamedAgg('transaction_dates', 'nunique'),
        debits_unique_transaction_months_last_6_months=pd.NamedAgg('year_month_transaction_dates', 'nunique')).reset_index()
    
   
    agent_summaries_last_6_months['debits_expected_transaction_days_last_6_months'] = (
                (max_transactions_date - (max_transactions_date + relativedelta(months=-6))) / np.timedelta64(1,
                                                                                                              'D') + 1)
    agent_summaries_last_6_months['debits_daily_trading_consistency_last_6_months'] = round(
        agent_summaries_last_6_months['debits_unique_transaction_days_last_6_months'] / agent_summaries_last_6_months[
            'debits_expected_transaction_days_last_6_months'], 2)

    agent_summaries_last_6_months['debits_average_daily_transactions_last_6_months'] = round(
        agent_summaries_last_6_months['debits_total_transactions_last_6_months'] / agent_summaries_last_6_months['debits_unique_transaction_days_last_6_months'])

    agent_summaries_last_6_months['debits_average_daily_debit_amt_last_6_months'] = round(
        agent_summaries_last_6_months['debits_total_debit_amount_last_6_months'] / agent_summaries_last_6_months[
            'debits_expected_transaction_days_last_6_months'])

    agent_summaries_last_6_months['debits_days_since_last_transaction_last_6_months'] = (
                (max_transactions_date - agent_summaries_last_6_months['debits_latest_transaction_date_last_6_months']) / np.timedelta64(1,
                                                                                                                    'D')).astype(
        int)

    agent_summaries_last_6_months = pd.merge(agent_summaries_last_6_months, negative_balances_summaries_last_6_months,
                                             on='terminal', how='left')

    

    agent_summaries_last_6_months['debits_lowest_negative_balance_last_6_months'].fillna(0, inplace=True)

    agent_summaries_last_6_months['debits_highest_negative_balance_last_6_months'].fillna(0, inplace=True)

    agent_summaries_last_6_months['debits_unique_negative_balance_dates_last_6_months'].fillna(0, inplace=True)

    agent_summaries_last_6_months['debits_earliest_negative_balance_date_last_6_months'].fillna(
        agent_summaries_last_6_months['debits_latest_transaction_date_last_6_months'], inplace=True)

    agent_summaries_last_6_months['debits_latest_negative_balance_date_last_6_months'].fillna(
        agent_summaries_last_6_months['debits_latest_transaction_date_last_6_months'], inplace=True)



    agent_summaries_last_6_months['debits_days_since_latest_negative_balance_last_6_months'] = ((agent_summaries_last_6_months[
                                                                                'debits_latest_transaction_date_last_6_months'] -
                                                                            agent_summaries_last_6_months[
                                                                                'debits_latest_negative_balance_date_last_6_months']) / np.timedelta64(
        1, 'D')).astype(int)




    return agent_summaries_last_6_months


In [35]:
# agent_summarries_last_6_monthss = calculate_6_months_debits_scoring_summaries()
# agent_summarries_last_6_monthss

In [36]:
def calculate_3_months_debits_scoring_summaries(data):
#     df=process_dates()
    df =data
    
    max_transactions_date = pd.to_datetime('today')
    #max_transactions_date = df['transaction_dates'].max()

    df_last_3_months = df[df['transaction_dates'] >= max_transactions_date + relativedelta(months=-3)]
    
    columns_to_convert = ['credit_amt', 'debit_amt','balance']

    for column in columns_to_convert:
        df_last_3_months[column] = pd.to_numeric(df_last_3_months[column], errors='coerce')
        df_last_3_months[column].fillna(0,inplace=True)


    debits_df_last_3_months = df_last_3_months[
        (df_last_3_months['debit_amt'] > 0) & (df_last_3_months['status'] == 'Approved')]

    debits_df_last_3_months_negative_balances = debits_df_last_3_months[debits_df_last_3_months['balance'] < 0]

    negative_balances_summaries_last_3_months = debits_df_last_3_months_negative_balances.groupby('terminal').agg(
        debits_lowest_negative_balance_last_3_months=pd.NamedAgg('balance', min),
        debits_highest_negative_balance_last_3_months=pd.NamedAgg('balance', max),
        debits_earliest_negative_balance_date_last_3_months=pd.NamedAgg('transaction_dates', min),
        debits_latest_negative_balance_date_last_3_months=pd.NamedAgg('transaction_dates', max),
        debits_unique_negative_balance_dates_last_3_months=pd.NamedAgg('transaction_dates', 'nunique')).reset_index()
    

    debits_df_last_3_months_without_negative_balances = debits_df_last_3_months[debits_df_last_3_months['balance'] >= 0]

    agent_summaries_last_3_months = debits_df_last_3_months_without_negative_balances.groupby('terminal').agg(
        debits_total_debit_amount_last_3_months=pd.NamedAgg('debit_amt', sum),
        debits_average_transaction_size_last_3_months=pd.NamedAgg('debit_amt', 'mean'),
        debits_total_transactions_last_3_months=pd.NamedAgg('request_ref', 'nunique'),
        debits_earliest_transaction_date_last_3_months=pd.NamedAgg('transaction_dates', min),
        debits_latest_transaction_date_last_3_months=pd.NamedAgg('transaction_dates', max),
        debits_no_of_services_offered_last_3_months=pd.NamedAgg('biller', 'nunique'),
        debits_min_balance_last_3_months=pd.NamedAgg('balance', min),
        debits_max_balance_last_3_months=pd.NamedAgg('balance', max),
        debits_average_balance_last_3_months=pd.NamedAgg('balance', 'mean'),
        debits_min_debit_amt_last_3_months=pd.NamedAgg('debit_amt', min),
        debits_max_debit_amt_last_3_months=pd.NamedAgg('debit_amt', max),
        debits_unique_transaction_days_last_3_months=pd.NamedAgg('transaction_dates', 'nunique'),
        debits_unique_transaction_months_last_3_months=pd.NamedAgg('year_month_transaction_dates', 'nunique')).reset_index()
    
   
    agent_summaries_last_3_months['debits_expected_transaction_days_last_3_months'] = (
                (max_transactions_date - (max_transactions_date + relativedelta(months=-3))) / np.timedelta64(1,
                                                                                                              'D') + 1)
    agent_summaries_last_3_months['debits_daily_trading_consistency_last_3_months'] = round(
        agent_summaries_last_3_months['debits_unique_transaction_days_last_3_months'] / agent_summaries_last_3_months[
            'debits_expected_transaction_days_last_3_months'], 2)

    agent_summaries_last_3_months['debits_average_daily_transactions_last_3_months'] = round(
        agent_summaries_last_3_months['debits_total_transactions_last_3_months'] / agent_summaries_last_3_months['debits_unique_transaction_days_last_3_months'])

    agent_summaries_last_3_months['debits_average_daily_debit_amt_last_3_months'] = round(
        agent_summaries_last_3_months['debits_total_debit_amount_last_3_months'] / agent_summaries_last_3_months[
            'debits_expected_transaction_days_last_3_months'])

    agent_summaries_last_3_months['debits_days_since_last_transaction_last_3_months'] = (
                (max_transactions_date - agent_summaries_last_3_months['debits_latest_transaction_date_last_3_months']) / np.timedelta64(1,
                                                                                                                    'D')).astype(
        int)

    agent_summaries_last_3_months = pd.merge(agent_summaries_last_3_months, negative_balances_summaries_last_3_months,
                                             on='terminal', how='left')

    

    agent_summaries_last_3_months['debits_lowest_negative_balance_last_3_months'].fillna(0, inplace=True)

    agent_summaries_last_3_months['debits_highest_negative_balance_last_3_months'].fillna(0, inplace=True)

    agent_summaries_last_3_months['debits_unique_negative_balance_dates_last_3_months'].fillna(0, inplace=True)

    agent_summaries_last_3_months['debits_earliest_negative_balance_date_last_3_months'].fillna(
        agent_summaries_last_3_months['debits_latest_transaction_date_last_3_months'], inplace=True)

    agent_summaries_last_3_months['debits_latest_negative_balance_date_last_3_months'].fillna(
        agent_summaries_last_3_months['debits_latest_transaction_date_last_3_months'], inplace=True)



    agent_summaries_last_3_months['debits_days_since_latest_negative_balance_last_3_months'] = ((agent_summaries_last_3_months[
                                                                                'debits_latest_transaction_date_last_3_months'] -
                                                                            agent_summaries_last_3_months[
                                                                                'debits_latest_negative_balance_date_last_3_months']) / np.timedelta64(
        1, 'D')).astype(int)




    return agent_summaries_last_3_months


In [37]:
# agent_summarries_last_3_monthss = calculate_3_months_debits_scoring_summaries()
# agent_summarries_last_3_monthss

In [38]:
def calculate_9_months_debits_scoring_summaries(data):
#     df=process_dates()
    df=data
    
    max_transactions_date = pd.to_datetime('today')
    #max_transactions_date = df['transaction_dates'].max()

    df_last_9_months = df[df['transaction_dates'] >= max_transactions_date + relativedelta(months=-9)]
    
    columns_to_convert = ['credit_amt', 'debit_amt','balance']

    for column in columns_to_convert:
        df_last_9_months[column] = pd.to_numeric(df_last_9_months[column], errors='coerce')
        df_last_9_months[column].fillna(0,inplace=True)


    debits_df_last_9_months = df_last_9_months[
        (df_last_9_months['debit_amt'] > 0) & (df_last_9_months['status'] == 'Approved')]

    debits_df_last_9_months_negative_balances = debits_df_last_9_months[debits_df_last_9_months['balance'] < 0]

    negative_balances_summaries_last_9_months = debits_df_last_9_months_negative_balances.groupby('terminal').agg(
        debits_lowest_negative_balance_last_9_months=pd.NamedAgg('balance', min),
        debits_highest_negative_balance_last_9_months=pd.NamedAgg('balance', max),
        debits_earliest_negative_balance_date_last_9_months=pd.NamedAgg('transaction_dates', min),
        debits_latest_negative_balance_date_last_9_months=pd.NamedAgg('transaction_dates', max),
        debits_unique_negative_balance_dates_last_9_months=pd.NamedAgg('transaction_dates', 'nunique')).reset_index()
    

    debits_df_last_9_months_without_negative_balances = debits_df_last_9_months[debits_df_last_9_months['balance'] >= 0]

    agent_summaries_last_9_months = debits_df_last_9_months_without_negative_balances.groupby('terminal').agg(
        debits_total_debit_amount_last_9_months=pd.NamedAgg('debit_amt', sum),
        debits_average_transaction_size_last_9_months=pd.NamedAgg('debit_amt', 'mean'),
        debits_total_transactions_last_9_months=pd.NamedAgg('request_ref', 'nunique'),
        debits_earliest_transaction_date_last_9_months=pd.NamedAgg('transaction_dates', min),
        debits_latest_transaction_date_last_9_months=pd.NamedAgg('transaction_dates', max),
        debits_no_of_services_offered_last_9_months=pd.NamedAgg('biller', 'nunique'),
        debits_min_balance_last_9_months=pd.NamedAgg('balance', min),
        debits_max_balance_last_9_months=pd.NamedAgg('balance', max),
        debits_average_balance_last_9_months=pd.NamedAgg('balance', 'mean'),
        debits_min_debit_amt_last_9_months=pd.NamedAgg('debit_amt', min),
        debits_max_debit_amt_last_9_months=pd.NamedAgg('debit_amt', max),
        debits_unique_transaction_days_last_9_months=pd.NamedAgg('transaction_dates', 'nunique'),
        debits_unique_transaction_months_last_9_months=pd.NamedAgg('year_month_transaction_dates', 'nunique')).reset_index()
    
   
    agent_summaries_last_9_months['debits_expected_transaction_days_last_9_months'] = (
                (max_transactions_date - (max_transactions_date + relativedelta(months=-9))) / np.timedelta64(1,
                                                                                                              'D') + 1)
    agent_summaries_last_9_months['debits_daily_trading_consistency_last_9_months'] = round(
        agent_summaries_last_9_months['debits_unique_transaction_days_last_9_months'] / agent_summaries_last_9_months[
            'debits_expected_transaction_days_last_9_months'], 2)

    agent_summaries_last_9_months['debits_average_daily_transactions_last_9_months'] = round(
        agent_summaries_last_9_months['debits_total_transactions_last_9_months'] / agent_summaries_last_9_months['debits_unique_transaction_days_last_9_months'])

    agent_summaries_last_9_months['debits_average_daily_debit_amt_last_9_months'] = round(
        agent_summaries_last_9_months['debits_total_debit_amount_last_9_months'] / agent_summaries_last_9_months[
            'debits_expected_transaction_days_last_9_months'])

    agent_summaries_last_9_months['debits_days_since_last_transaction_last_9_months'] = (
                (max_transactions_date - agent_summaries_last_9_months['debits_latest_transaction_date_last_9_months']) / np.timedelta64(1,
                                                                                                                    'D')).astype(
        int)

    agent_summaries_last_9_months = pd.merge(agent_summaries_last_9_months, negative_balances_summaries_last_9_months,
                                             on='terminal', how='left')

    

    agent_summaries_last_9_months['debits_lowest_negative_balance_last_9_months'].fillna(0, inplace=True)

    agent_summaries_last_9_months['debits_highest_negative_balance_last_9_months'].fillna(0, inplace=True)

    agent_summaries_last_9_months['debits_unique_negative_balance_dates_last_9_months'].fillna(0, inplace=True)

    agent_summaries_last_9_months['debits_earliest_negative_balance_date_last_9_months'].fillna(
        agent_summaries_last_9_months['debits_latest_transaction_date_last_9_months'], inplace=True)

    agent_summaries_last_9_months['debits_latest_negative_balance_date_last_9_months'].fillna(
        agent_summaries_last_9_months['debits_latest_transaction_date_last_9_months'], inplace=True)



    agent_summaries_last_9_months['debits_days_since_latest_negative_balance_last_9_months'] = ((agent_summaries_last_9_months[
                                                                                'debits_latest_transaction_date_last_9_months'] -
                                                                            agent_summaries_last_9_months[
                                                                                'debits_latest_negative_balance_date_last_9_months']) / np.timedelta64(
        1, 'D')).astype(int)




    return agent_summaries_last_9_months


In [39]:
# agent_summarries_last_9_monthss = calculate_9_months_debits_scoring_summaries()
# agent_summarries_last_9_monthss

In [40]:
def calculate_12_months_debits_scoring_summaries(data):
#     df=process_dates()
    df=data
    
    max_transactions_date = pd.to_datetime('today')
    #max_transactions_date = df['transaction_dates'].max()

    df_last_12_months = df[df['transaction_dates'] >= max_transactions_date + relativedelta(months=-12)]
    
    columns_to_convert = ['credit_amt', 'debit_amt','balance']

    for column in columns_to_convert:
        df_last_12_months[column] = pd.to_numeric(df_last_12_months[column], errors='coerce')
        df_last_12_months[column].fillna(0,inplace=True)


    debits_df_last_12_months = df_last_12_months[
        (df_last_12_months['debit_amt'] > 0) & (df_last_12_months['status'] == 'Approved')]

    debits_df_last_12_months_negative_balances = debits_df_last_12_months[debits_df_last_12_months['balance'] < 0]

    negative_balances_summaries_last_12_months = debits_df_last_12_months_negative_balances.groupby('terminal').agg(
        debits_lowest_negative_balance_last_12_months=pd.NamedAgg('balance', min),
        debits_highest_negative_balance_last_12_months=pd.NamedAgg('balance', max),
        debits_earliest_negative_balance_date_last_12_months=pd.NamedAgg('transaction_dates', min),
        debits_latest_negative_balance_date_last_12_months=pd.NamedAgg('transaction_dates', max),
        debits_unique_negative_balance_dates_last_12_months=pd.NamedAgg('transaction_dates', 'nunique')).reset_index()
    

    debits_df_last_12_months_without_negative_balances = debits_df_last_12_months[debits_df_last_12_months['balance'] >= 0]

    agent_summaries_last_12_months = debits_df_last_12_months_without_negative_balances.groupby('terminal').agg(
        debits_total_debit_amount_last_12_months=pd.NamedAgg('debit_amt', sum),
        debits_average_transaction_size_last_12_months=pd.NamedAgg('debit_amt', 'mean'),
        debits_total_transactions_last_12_months=pd.NamedAgg('request_ref', 'nunique'),
        debits_earliest_transaction_date_last_12_months=pd.NamedAgg('transaction_dates', min),
        debits_latest_transaction_date_last_12_months=pd.NamedAgg('transaction_dates', max),
        debits_no_of_services_offered_last_12_months=pd.NamedAgg('biller', 'nunique'),
        debits_min_balance_last_12_months=pd.NamedAgg('balance', min),
        debits_max_balance_last_12_months=pd.NamedAgg('balance', max),
        debits_average_balance_last_12_months=pd.NamedAgg('balance', 'mean'),
        debits_min_debit_amt_last_12_months=pd.NamedAgg('debit_amt', min),
        debits_max_debit_amt_last_12_months=pd.NamedAgg('debit_amt', max),
        debits_unique_transaction_days_last_12_months=pd.NamedAgg('transaction_dates', 'nunique'),
        debits_unique_transaction_months_last_12_months=pd.NamedAgg('year_month_transaction_dates', 'nunique')).reset_index()
    
   
    agent_summaries_last_12_months['debits_expected_transaction_days_last_12_months'] = (
                (max_transactions_date - (max_transactions_date + relativedelta(months=-12))) / np.timedelta64(1,
                                                                                                              'D') + 1)
    agent_summaries_last_12_months['debits_daily_trading_consistency_last_12_months'] = round(
        agent_summaries_last_12_months['debits_unique_transaction_days_last_12_months'] / agent_summaries_last_12_months[
            'debits_expected_transaction_days_last_12_months'], 2)

    agent_summaries_last_12_months['debits_average_daily_transactions_last_12_months'] = round(
        agent_summaries_last_12_months['debits_total_transactions_last_12_months'] / agent_summaries_last_12_months['debits_unique_transaction_days_last_12_months'])

    agent_summaries_last_12_months['debits_average_daily_debit_amt_last_12_months'] = round(
        agent_summaries_last_12_months['debits_total_debit_amount_last_12_months'] / agent_summaries_last_12_months[
            'debits_expected_transaction_days_last_12_months'])

    agent_summaries_last_12_months['debits_days_since_last_transaction_last_12_months'] = (
                (max_transactions_date - agent_summaries_last_12_months['debits_latest_transaction_date_last_12_months']) / np.timedelta64(1,
                                                                                                                    'D')).astype(
        int)

    agent_summaries_last_12_months = pd.merge(agent_summaries_last_12_months, negative_balances_summaries_last_12_months,
                                             on='terminal', how='left')

    

    agent_summaries_last_12_months['debits_lowest_negative_balance_last_12_months'].fillna(0, inplace=True)

    agent_summaries_last_12_months['debits_highest_negative_balance_last_12_months'].fillna(0, inplace=True)

    agent_summaries_last_12_months['debits_unique_negative_balance_dates_last_12_months'].fillna(0, inplace=True)

    agent_summaries_last_12_months['debits_earliest_negative_balance_date_last_12_months'].fillna(
        agent_summaries_last_12_months['debits_latest_transaction_date_last_12_months'], inplace=True)

    agent_summaries_last_12_months['debits_latest_negative_balance_date_last_12_months'].fillna(
        agent_summaries_last_12_months['debits_latest_transaction_date_last_12_months'], inplace=True)



    agent_summaries_last_12_months['debits_days_since_latest_negative_balance_last_12_months'] = ((agent_summaries_last_12_months[
                                                                                'debits_latest_transaction_date_last_12_months'] -
                                                                            agent_summaries_last_12_months[
                                                                                'debits_latest_negative_balance_date_last_12_months']) / np.timedelta64(
        1, 'D')).astype(int)




    return agent_summaries_last_12_months


In [41]:
# agent_summarries_last_12_monthss = calculate_12_months_debits_scoring_summaries()
# agent_summarries_last_12_monthss

In [42]:
def calculate_1_months_debits_scoring_summaries(data):
#     df=process_dates()
    df=data
    
    max_transactions_date = pd.to_datetime('today')
    #max_transactions_date = df['transaction_dates'].max()

    df_last_1_months = df[df['transaction_dates'] >= max_transactions_date + relativedelta(months=-1)]
    
    columns_to_convert = ['credit_amt', 'debit_amt','balance']

    for column in columns_to_convert:
        df_last_1_months[column] = pd.to_numeric(df_last_1_months[column], errors='coerce')
        df_last_1_months[column].fillna(0,inplace=True)


    debits_df_last_1_months = df_last_1_months[
        (df_last_1_months['debit_amt'] > 0) & (df_last_1_months['status'] == 'Approved')]

    debits_df_last_1_months_negative_balances = debits_df_last_1_months[debits_df_last_1_months['balance'] < 0]

    negative_balances_summaries_last_1_months = debits_df_last_1_months_negative_balances.groupby('terminal').agg(
        debits_lowest_negative_balance_last_1_months=pd.NamedAgg('balance', min),
        debits_highest_negative_balance_last_1_months=pd.NamedAgg('balance', max),
        debits_earliest_negative_balance_date_last_1_months=pd.NamedAgg('transaction_dates', min),
        debits_latest_negative_balance_date_last_1_months=pd.NamedAgg('transaction_dates', max),
        debits_unique_negative_balance_dates_last_1_months=pd.NamedAgg('transaction_dates', 'nunique')).reset_index()
    

    debits_df_last_1_months_without_negative_balances = debits_df_last_1_months[debits_df_last_1_months['balance'] >= 0]

    agent_summaries_last_1_months = debits_df_last_1_months_without_negative_balances.groupby('terminal').agg(
        debits_total_debit_amount_last_1_months=pd.NamedAgg('debit_amt', sum),
        debits_average_transaction_size_last_1_months=pd.NamedAgg('debit_amt', 'mean'),
        debits_total_transactions_last_1_months=pd.NamedAgg('request_ref', 'nunique'),
        debits_earliest_transaction_date_last_1_months=pd.NamedAgg('transaction_dates', min),
        debits_latest_transaction_date_last_1_months=pd.NamedAgg('transaction_dates', max),
        debits_no_of_services_offered_last_1_months=pd.NamedAgg('biller', 'nunique'),
        debits_min_balance_last_1_months=pd.NamedAgg('balance', min),
        debits_max_balance_last_1_months=pd.NamedAgg('balance', max),
        debits_average_balance_last_1_months=pd.NamedAgg('balance', 'mean'),
        debits_min_debit_amt_last_1_months=pd.NamedAgg('debit_amt', min),
        debits_max_debit_amt_last_1_months=pd.NamedAgg('debit_amt', max),
        debits_unique_transaction_days_last_1_months=pd.NamedAgg('transaction_dates', 'nunique'),
        debits_unique_transaction_months_last_1_months=pd.NamedAgg('year_month_transaction_dates', 'nunique')).reset_index()
    
   
    agent_summaries_last_1_months['debits_expected_transaction_days_last_1_months'] = (
                (max_transactions_date - (max_transactions_date + relativedelta(months=-1))) / np.timedelta64(1,
                                                                                                              'D') + 1)
    agent_summaries_last_1_months['debits_daily_trading_consistency_last_1_months'] = round(
        agent_summaries_last_1_months['debits_unique_transaction_days_last_1_months'] / agent_summaries_last_1_months[
            'debits_expected_transaction_days_last_1_months'], 2)

    agent_summaries_last_1_months['debits_average_daily_transactions_last_1_months'] = round(
        agent_summaries_last_1_months['debits_total_transactions_last_1_months'] / agent_summaries_last_1_months['debits_unique_transaction_days_last_1_months'])

    agent_summaries_last_1_months['debits_average_daily_debit_amt_last_1_months'] = round(
        agent_summaries_last_1_months['debits_total_debit_amount_last_1_months'] / agent_summaries_last_1_months[
            'debits_expected_transaction_days_last_1_months'])

    agent_summaries_last_1_months['debits_days_since_last_transaction_last_1_months'] = (
                (max_transactions_date - agent_summaries_last_1_months['debits_latest_transaction_date_last_1_months']) / np.timedelta64(1,
                                                                                                                    'D')).astype(
        int)

    agent_summaries_last_1_months = pd.merge(agent_summaries_last_1_months, negative_balances_summaries_last_1_months,
                                             on='terminal', how='left')

    

    agent_summaries_last_1_months['debits_lowest_negative_balance_last_1_months'].fillna(0, inplace=True)

    agent_summaries_last_1_months['debits_highest_negative_balance_last_1_months'].fillna(0, inplace=True)

    agent_summaries_last_1_months['debits_unique_negative_balance_dates_last_1_months'].fillna(0, inplace=True)

    agent_summaries_last_1_months['debits_earliest_negative_balance_date_last_1_months'].fillna(
        agent_summaries_last_1_months['debits_latest_transaction_date_last_1_months'], inplace=True)

    agent_summaries_last_1_months['debits_latest_negative_balance_date_last_1_months'].fillna(
        agent_summaries_last_1_months['debits_latest_transaction_date_last_1_months'], inplace=True)



    agent_summaries_last_1_months['debits_days_since_latest_negative_balance_last_1_months'] = ((agent_summaries_last_1_months[
                                                                                'debits_latest_transaction_date_last_1_months'] -
                                                                            agent_summaries_last_1_months[
                                                                                'debits_latest_negative_balance_date_last_1_months']) / np.timedelta64(
        1, 'D')).astype(int)




    return agent_summaries_last_1_months


In [43]:
# agent_summarries_last_1_monthss = calculate_1_months_debits_scoring_summaries()
# agent_summarries_last_1_monthss

### Credits Scoring Summaries

In [44]:
def calculate_6_months_credit_scoring_summaries(data):
#     df=process_dates()
    df=data
    
    max_transactions_date = pd.to_datetime('today')
    #max_transactions_date = df['transaction_dates'].max()

    df_last_6_months = df[df['transaction_dates'] >= max_transactions_date + relativedelta(months=-6)]
    columns_to_convert = ['credit_amt', 'debit_amt','balance']

    for column in columns_to_convert:
        df_last_6_months[column] = pd.to_numeric(df_last_6_months[column], errors='coerce')
        df_last_6_months[column].fillna(0,inplace=True)


   
    credits_df_last_6_months = df_last_6_months[
        (df_last_6_months['credit_amt'] > 0) & (df_last_6_months['status'] == 'Approved')]
    
    credits_df_last_6_months_negative_balances = credits_df_last_6_months[credits_df_last_6_months['balance'] < 0]
    
    credits_negative_balances_summaries_last_6_months = credits_df_last_6_months_negative_balances.groupby('terminal').agg(
        credits_lowest_negative_balance_last_6_months=pd.NamedAgg('balance', min),
        credits_highest_negative_balance_last_6_months=pd.NamedAgg('balance', max),
        credits_earliest_negative_balance_date_last_6_months=pd.NamedAgg('transaction_dates', min),
        credits_latest_negative_balance_date_last_6_months=pd.NamedAgg('transaction_dates', max),
        credits_unique_negative_balance_dates_last_6_months=pd.NamedAgg('transaction_dates', 'nunique')).reset_index()
    
    credits_df_last_6_months_without_negative_balances = credits_df_last_6_months[credits_df_last_6_months['balance'] >= 0]
    
    credits_agent_summaries_last_6_months = credits_df_last_6_months_without_negative_balances.groupby('terminal').agg(
        total_credit_amount_last_6_months=pd.NamedAgg('credit_amt', sum),
        credits_average_transaction_size_last_6_months=pd.NamedAgg('credit_amt', 'mean'),
        credits_total_transactions_last_6_months=pd.NamedAgg('request_ref', 'nunique'),
        credits_earliest_transaction_date_last_6_months=pd.NamedAgg('transaction_dates', min),
        credits_latest_transaction_date_last_6_months=pd.NamedAgg('transaction_dates', max),
        credits_no_of_services_offered_last_6_months=pd.NamedAgg('biller', 'nunique'),
        credits_min_balance_last_6_months=pd.NamedAgg('balance', min),
        credits_max_balance_last_6_months=pd.NamedAgg('balance', max),
        credits_average_balance_last_6_months=pd.NamedAgg('balance', 'mean'),
        min_credit_amt_last_6_months=pd.NamedAgg('credit_amt', min),
        max_credit_amt_last_6_months=pd.NamedAgg('credit_amt', max),
        credits_unique_transaction_days_last_6_months=pd.NamedAgg('transaction_dates', 'nunique'),
        credits_unique_transaction_months_last_6_months=pd.NamedAgg('year_month_transaction_dates', 'nunique')).reset_index()

    credits_agent_summaries_last_6_months['credits_expected_transaction_days_last_6_months'] = (
            (max_transactions_date - (max_transactions_date + relativedelta(months=-6))) / np.timedelta64(1,
                                                                                                          'D') + 1)
    credits_agent_summaries_last_6_months['credits_daily_trading_consistency_last_6_months'] = round(
    credits_agent_summaries_last_6_months['credits_unique_transaction_days_last_6_months'] / credits_agent_summaries_last_6_months[
        'credits_expected_transaction_days_last_6_months'], 2)

    credits_agent_summaries_last_6_months['credits_average_daily_transactions_last_6_months'] = round(
    credits_agent_summaries_last_6_months['credits_total_transactions_last_6_months'] / credits_agent_summaries_last_6_months['credits_unique_transaction_days_last_6_months'])

    credits_agent_summaries_last_6_months['average_daily_credit_amt_last_6_months'] = round(
    credits_agent_summaries_last_6_months['total_credit_amount_last_6_months'] / credits_agent_summaries_last_6_months[
        'credits_expected_transaction_days_last_6_months'])

    credits_agent_summaries_last_6_months['credits_days_since_last_transaction_last_6_months'] = (
            (max_transactions_date - credits_agent_summaries_last_6_months['credits_latest_transaction_date_last_6_months']) / np.timedelta64(1,
                                                                                                                'D')).astype(
    int)

    credits_agent_summaries_last_6_months = pd.merge(credits_agent_summaries_last_6_months, credits_negative_balances_summaries_last_6_months,
                                          on='terminal', how='left')


    





    credits_agent_summaries_last_6_months['credits_lowest_negative_balance_last_6_months'].fillna(0, inplace=True)

    credits_agent_summaries_last_6_months['credits_highest_negative_balance_last_6_months'].fillna(0, inplace=True)

    credits_agent_summaries_last_6_months['credits_unique_negative_balance_dates_last_6_months'].fillna(0, inplace=True)

    credits_agent_summaries_last_6_months['credits_earliest_negative_balance_date_last_6_months'].fillna(credits_agent_summaries_last_6_months['credits_latest_transaction_date_last_6_months'], inplace=True)

    credits_agent_summaries_last_6_months['credits_latest_negative_balance_date_last_6_months'].fillna(credits_agent_summaries_last_6_months['credits_latest_transaction_date_last_6_months'], inplace=True)
    
    credits_agent_summaries_last_6_months['credits_days_since_latest_negative_balance_last_6_months'] = ((credits_agent_summaries_last_6_months[
                                                                                'credits_latest_transaction_date_last_6_months'] -
                                                                            credits_agent_summaries_last_6_months[
                                                                                'credits_latest_negative_balance_date_last_6_months']) / np.timedelta64(
        1, 'D')).astype(int)
    

    
    return credits_agent_summaries_last_6_months
#     return credits_agent_summaries_last_6_months

In [45]:
# credit_summaries_last_6_months= calculate_6_months_credit_scoring_summaries()
# credit_summaries_last_6_months

In [46]:
def calculate_12_months_credit_scoring_summaries(data):
#     df=process_dates()
    df=data
    
    
    max_transactions_date = pd.to_datetime('today')
    #max_transactions_date = df['transaction_dates'].max()

    df_last_12_months = df[df['transaction_dates'] >= max_transactions_date + relativedelta(months=-12)]
    columns_to_convert = ['credit_amt', 'debit_amt','balance']

    for column in columns_to_convert:
        df_last_12_months[column] = pd.to_numeric(df_last_12_months[column], errors='coerce')
        df_last_12_months[column].fillna(0,inplace=True)


   
    credits_df_last_12_months = df_last_12_months[
        (df_last_12_months['credit_amt'] > 0) & (df_last_12_months['status'] == 'Approved')]
    
    credits_df_last_12_months_negative_balances = credits_df_last_12_months[credits_df_last_12_months['balance'] < 0]
    
    credits_negative_balances_summaries_last_12_months = credits_df_last_12_months_negative_balances.groupby('terminal').agg(
        credits_lowest_negative_balance_last_12_months=pd.NamedAgg('balance', min),
        credits_highest_negative_balance_last_12_months=pd.NamedAgg('balance', max),
        credits_earliest_negative_balance_date_last_12_months=pd.NamedAgg('transaction_dates', min),
        credits_latest_negative_balance_date_last_12_months=pd.NamedAgg('transaction_dates', max),
        credits_unique_negative_balance_dates_last_12_months=pd.NamedAgg('transaction_dates', 'nunique')).reset_index()
    
    credits_df_last_12_months_without_negative_balances = credits_df_last_12_months[credits_df_last_12_months['balance'] >= 0]
    
    credits_agent_summaries_last_12_months = credits_df_last_12_months_without_negative_balances.groupby('terminal').agg(
        total_credit_amount_last_12_months=pd.NamedAgg('credit_amt', sum),
        credits_average_transaction_size_last_12_months=pd.NamedAgg('credit_amt', 'mean'),
        credits_total_transactions_last_12_months=pd.NamedAgg('request_ref', 'nunique'),
        credits_earliest_transaction_date_last_12_months=pd.NamedAgg('transaction_dates', min),
        credits_latest_transaction_date_last_12_months=pd.NamedAgg('transaction_dates', max),
        credits_no_of_services_offered_last_12_months=pd.NamedAgg('biller', 'nunique'),
        credits_min_balance_last_12_months=pd.NamedAgg('balance', min),
        credits_max_balance_last_12_months=pd.NamedAgg('balance', max),
        credits_average_balance_last_12_months=pd.NamedAgg('balance', 'mean'),
        min_credit_amt_last_12_months=pd.NamedAgg('credit_amt', min),
        max_credit_amt_last_12_months=pd.NamedAgg('credit_amt', max),
        credits_unique_transaction_days_last_12_months=pd.NamedAgg('transaction_dates', 'nunique'),
        credits_unique_transaction_months_last_12_months=pd.NamedAgg('year_month_transaction_dates', 'nunique')).reset_index()

    credits_agent_summaries_last_12_months['credits_expected_transaction_days_last_12_months'] = (
            (max_transactions_date - (max_transactions_date + relativedelta(months=-12))) / np.timedelta64(1,
                                                                                                          'D') + 1)
    credits_agent_summaries_last_12_months['credits_daily_trading_consistency_last_12_months'] = round(
    credits_agent_summaries_last_12_months['credits_unique_transaction_days_last_12_months'] / credits_agent_summaries_last_12_months[
        'credits_expected_transaction_days_last_12_months'], 2)

    credits_agent_summaries_last_12_months['credits_average_daily_transactions_last_12_months'] = round(
    credits_agent_summaries_last_12_months['credits_total_transactions_last_12_months'] / credits_agent_summaries_last_12_months['credits_unique_transaction_days_last_12_months'])

    credits_agent_summaries_last_12_months['average_daily_credit_amt_last_12_months'] = round(
    credits_agent_summaries_last_12_months['total_credit_amount_last_12_months'] / credits_agent_summaries_last_12_months[
        'credits_expected_transaction_days_last_12_months'])

    credits_agent_summaries_last_12_months['credits_days_since_last_transaction_last_12_months'] = (
            (max_transactions_date - credits_agent_summaries_last_12_months['credits_latest_transaction_date_last_12_months']) / np.timedelta64(1,
                                                                                                                'D')).astype(
    int)

    credits_agent_summaries_last_12_months = pd.merge(credits_agent_summaries_last_12_months, credits_negative_balances_summaries_last_12_months,
                                          on='terminal', how='left')


    





    credits_agent_summaries_last_12_months['credits_lowest_negative_balance_last_12_months'].fillna(0, inplace=True)

    credits_agent_summaries_last_12_months['credits_highest_negative_balance_last_12_months'].fillna(0, inplace=True)

    credits_agent_summaries_last_12_months['credits_unique_negative_balance_dates_last_12_months'].fillna(0, inplace=True)

    credits_agent_summaries_last_12_months['credits_earliest_negative_balance_date_last_12_months'].fillna(credits_agent_summaries_last_12_months['credits_latest_transaction_date_last_12_months'], inplace=True)

    credits_agent_summaries_last_12_months['credits_latest_negative_balance_date_last_12_months'].fillna(credits_agent_summaries_last_12_months['credits_latest_transaction_date_last_12_months'], inplace=True)
    
    credits_agent_summaries_last_12_months['credits_days_since_latest_negative_balance_last_12_months'] = ((credits_agent_summaries_last_12_months[
                                                                                'credits_latest_transaction_date_last_12_months'] -
                                                                            credits_agent_summaries_last_12_months[
                                                                                'credits_latest_negative_balance_date_last_12_months']) / np.timedelta64(
        1, 'D')).astype(int)
    

    
    return credits_agent_summaries_last_12_months
#     return credits_agent_summaries_last_6_months

In [47]:
# credit_summaries_last_12_months= calculate_12_months_credit_scoring_summaries()
# credit_summaries_last_12_months

In [48]:
def calculate_3_months_credit_scoring_summaries(data):
#     df=process_dates()
    df=data
    
    max_transactions_date = pd.to_datetime('today')
    #max_transactions_date = df['transaction_dates'].max()

    df_last_3_months = df[df['transaction_dates'] >= max_transactions_date + relativedelta(months=-3)]
    columns_to_convert = ['credit_amt', 'debit_amt','balance']

    for column in columns_to_convert:
        df_last_3_months[column] = pd.to_numeric(df_last_3_months[column], errors='coerce')
        df_last_3_months[column].fillna(0,inplace=True)


   
    credits_df_last_3_months = df_last_3_months[
        (df_last_3_months['credit_amt'] > 0) & (df_last_3_months['status'] == 'Approved')]
    
    credits_df_last_3_months_negative_balances = credits_df_last_3_months[credits_df_last_3_months['balance'] < 0]
    
    credits_negative_balances_summaries_last_3_months = credits_df_last_3_months_negative_balances.groupby('terminal').agg(
        credits_lowest_negative_balance_last_3_months=pd.NamedAgg('balance', min),
        credits_highest_negative_balance_last_3_months=pd.NamedAgg('balance', max),
        credits_earliest_negative_balance_date_last_3_months=pd.NamedAgg('transaction_dates', min),
        credits_latest_negative_balance_date_last_3_months=pd.NamedAgg('transaction_dates', max),
        credits_unique_negative_balance_dates_last_3_months=pd.NamedAgg('transaction_dates', 'nunique')).reset_index()
    
    credits_df_last_3_months_without_negative_balances = credits_df_last_3_months[credits_df_last_3_months['balance'] >= 0]
    
    credits_agent_summaries_last_3_months = credits_df_last_3_months_without_negative_balances.groupby('terminal').agg(
        total_credit_amount_last_3_months=pd.NamedAgg('credit_amt', sum),
        credits_average_transaction_size_last_3_months=pd.NamedAgg('credit_amt', 'mean'),
        credits_total_transactions_last_3_months=pd.NamedAgg('request_ref', 'nunique'),
        credits_earliest_transaction_date_last_3_months=pd.NamedAgg('transaction_dates', min),
        credits_latest_transaction_date_last_3_months=pd.NamedAgg('transaction_dates', max),
        credits_no_of_services_offered_last_3_months=pd.NamedAgg('biller', 'nunique'),
        credits_min_balance_last_3_months=pd.NamedAgg('balance', min),
        credits_max_balance_last_3_months=pd.NamedAgg('balance', max),
        credits_average_balance_last_3_months=pd.NamedAgg('balance', 'mean'),
        min_credit_amt_last_3_months=pd.NamedAgg('credit_amt', min),
        max_credit_amt_last_3_months=pd.NamedAgg('credit_amt', max),
        credits_unique_transaction_days_last_3_months=pd.NamedAgg('transaction_dates', 'nunique'),
        credits_unique_transaction_months_last_3_months=pd.NamedAgg('year_month_transaction_dates', 'nunique')).reset_index()

    credits_agent_summaries_last_3_months['credits_expected_transaction_days_last_3_months'] = (
            (max_transactions_date - (max_transactions_date + relativedelta(months=-9))) / np.timedelta64(1,
                                                                                                          'D') + 1)
    credits_agent_summaries_last_3_months['credits_daily_trading_consistency_last_3_months'] = round(
    credits_agent_summaries_last_3_months['credits_unique_transaction_days_last_3_months'] / credits_agent_summaries_last_3_months[
        'credits_expected_transaction_days_last_3_months'], 2)

    credits_agent_summaries_last_3_months['credits_average_daily_transactions_last_3_months'] = round(
    credits_agent_summaries_last_3_months['credits_total_transactions_last_3_months'] / credits_agent_summaries_last_3_months['credits_unique_transaction_days_last_3_months'])

    credits_agent_summaries_last_3_months['average_daily_credit_amt_last_3_months'] = round(
    credits_agent_summaries_last_3_months['total_credit_amount_last_3_months'] / credits_agent_summaries_last_3_months[
        'credits_expected_transaction_days_last_3_months'])

    credits_agent_summaries_last_3_months['credits_days_since_last_transaction_last_3_months'] = (
            (max_transactions_date - credits_agent_summaries_last_3_months['credits_latest_transaction_date_last_3_months']) / np.timedelta64(1,
                                                                                                                'D')).astype(
    int)

    credits_agent_summaries_last_3_months = pd.merge(credits_agent_summaries_last_3_months, credits_negative_balances_summaries_last_3_months,
                                          on='terminal', how='left')


    





    credits_agent_summaries_last_3_months['credits_lowest_negative_balance_last_3_months'].fillna(0, inplace=True)

    credits_agent_summaries_last_3_months['credits_highest_negative_balance_last_3_months'].fillna(0, inplace=True)

    credits_agent_summaries_last_3_months['credits_unique_negative_balance_dates_last_3_months'].fillna(0, inplace=True)

    credits_agent_summaries_last_3_months['credits_earliest_negative_balance_date_last_3_months'].fillna(credits_agent_summaries_last_3_months['credits_latest_transaction_date_last_3_months'], inplace=True)

    credits_agent_summaries_last_3_months['credits_latest_negative_balance_date_last_3_months'].fillna(credits_agent_summaries_last_3_months['credits_latest_transaction_date_last_3_months'], inplace=True)
    
    credits_agent_summaries_last_3_months['credits_days_since_latest_negative_balance_last_3_months'] = ((credits_agent_summaries_last_3_months[
                                                                                'credits_latest_transaction_date_last_3_months'] -
                                                                            credits_agent_summaries_last_3_months[
                                                                                'credits_latest_negative_balance_date_last_3_months']) / np.timedelta64(
        1, 'D')).astype(int)
    

    
    return credits_agent_summaries_last_3_months
#     return credits_agent_summaries_last_6_months

In [49]:
# credit_summaries_last_3_months= calculate_3_months_credit_scoring_summaries()
# credit_summaries_last_3_months

In [50]:
def calculate_1_months_credit_scoring_summaries(data):
#     df=process_dates()
    df=data
    
    max_transactions_date = pd.to_datetime('today')
    #max_transactions_date = df['transaction_dates'].max()

    df_last_1_months = df[df['transaction_dates'] >= max_transactions_date + relativedelta(months=-1)]
    columns_to_convert = ['credit_amt', 'debit_amt','balance']

    for column in columns_to_convert:
        df_last_1_months[column] = pd.to_numeric(df_last_1_months[column], errors='coerce')
        df_last_1_months[column].fillna(0,inplace=True)


   
    credits_df_last_1_months = df_last_1_months[
        (df_last_1_months['credit_amt'] > 0) & (df_last_1_months['status'] == 'Approved')]
    
    credits_df_last_1_months_negative_balances = credits_df_last_1_months[credits_df_last_1_months['balance'] < 0]
    
    credits_negative_balances_summaries_last_1_months = credits_df_last_1_months_negative_balances.groupby('terminal').agg(
        credits_lowest_negative_balance_last_1_months=pd.NamedAgg('balance', min),
        credits_highest_negative_balance_last_1_months=pd.NamedAgg('balance', max),
        credits_earliest_negative_balance_date_last_1_months=pd.NamedAgg('transaction_dates', min),
        credits_latest_negative_balance_date_last_1_months=pd.NamedAgg('transaction_dates', max),
        credits_unique_negative_balance_dates_last_1_months=pd.NamedAgg('transaction_dates', 'nunique')).reset_index()
    
    credits_df_last_1_months_without_negative_balances = credits_df_last_1_months[credits_df_last_1_months['balance'] >= 0]
    
    credits_agent_summaries_last_1_months = credits_df_last_1_months_without_negative_balances.groupby('terminal').agg(
        total_credit_amount_last_1_months=pd.NamedAgg('credit_amt', sum),
        credits_average_transaction_size_last_1_months=pd.NamedAgg('credit_amt', 'mean'),
        credits_total_transactions_last_1_months=pd.NamedAgg('request_ref', 'nunique'),
        credits_earliest_transaction_date_last_1_months=pd.NamedAgg('transaction_dates', min),
        credits_latest_transaction_date_last_1_months=pd.NamedAgg('transaction_dates', max),
        credits_no_of_services_offered_last_1_months=pd.NamedAgg('biller', 'nunique'),
        credits_min_balance_last_1_months=pd.NamedAgg('balance', min),
        credits_max_balance_last_1_months=pd.NamedAgg('balance', max),
        credits_average_balance_last_1_months=pd.NamedAgg('balance', 'mean'),
        min_credit_amt_last_1_months=pd.NamedAgg('credit_amt', min),
        max_credit_amt_last_1_months=pd.NamedAgg('credit_amt', max),
        credits_unique_transaction_days_last_1_months=pd.NamedAgg('transaction_dates', 'nunique'),
        credits_unique_transaction_months_last_1_months=pd.NamedAgg('year_month_transaction_dates', 'nunique')).reset_index()

    credits_agent_summaries_last_1_months['credits_expected_transaction_days_last_1_months'] = (
            (max_transactions_date - (max_transactions_date + relativedelta(months=-1))) / np.timedelta64(1,
                                                                                                          'D') + 1)
    credits_agent_summaries_last_1_months['credits_daily_trading_consistency_last_1_months'] = round(
    credits_agent_summaries_last_1_months['credits_unique_transaction_days_last_1_months'] / credits_agent_summaries_last_1_months[
        'credits_expected_transaction_days_last_1_months'], 2)

    credits_agent_summaries_last_1_months['credits_average_daily_transactions_last_1_months'] = round(
    credits_agent_summaries_last_1_months['credits_total_transactions_last_1_months'] / credits_agent_summaries_last_1_months['credits_unique_transaction_days_last_1_months'])

    credits_agent_summaries_last_1_months['average_daily_credit_amt_last_1_months'] = round(
    credits_agent_summaries_last_1_months['total_credit_amount_last_1_months'] / credits_agent_summaries_last_1_months[
        'credits_expected_transaction_days_last_1_months'])

    credits_agent_summaries_last_1_months['credits_days_since_last_transaction_last_1_months'] = (
            (max_transactions_date - credits_agent_summaries_last_1_months['credits_latest_transaction_date_last_1_months']) / np.timedelta64(1,
                                                                                                                'D')).astype(
    int)

    credits_agent_summaries_last_1_months = pd.merge(credits_agent_summaries_last_1_months, credits_negative_balances_summaries_last_1_months,
                                          on='terminal', how='left')


    





    credits_agent_summaries_last_1_months['credits_lowest_negative_balance_last_1_months'].fillna(0, inplace=True)

    credits_agent_summaries_last_1_months['credits_highest_negative_balance_last_1_months'].fillna(0, inplace=True)

    credits_agent_summaries_last_1_months['credits_unique_negative_balance_dates_last_1_months'].fillna(0, inplace=True)

    credits_agent_summaries_last_1_months['credits_earliest_negative_balance_date_last_1_months'].fillna(credits_agent_summaries_last_1_months['credits_latest_transaction_date_last_1_months'], inplace=True)

    credits_agent_summaries_last_1_months['credits_latest_negative_balance_date_last_1_months'].fillna(credits_agent_summaries_last_1_months['credits_latest_transaction_date_last_1_months'], inplace=True)
    
    credits_agent_summaries_last_1_months['credits_days_since_latest_negative_balance_last_1_months'] = ((credits_agent_summaries_last_1_months[
                                                                                'credits_latest_transaction_date_last_1_months'] -
                                                                            credits_agent_summaries_last_1_months[
                                                                                'credits_latest_negative_balance_date_last_1_months']) / np.timedelta64(
        1, 'D')).astype(int)
    

    
    return credits_agent_summaries_last_1_months
#     return credits_agent_summaries_last_6_months

In [51]:
# credit_summaries_last_1_months= calculate_1_months_credit_scoring_summaries()
# credit_summaries_last_1_months

In [52]:
def calculate_9_months_credit_scoring_summaries(data):
    df=data
#     df=process_dates()
    
    max_transactions_date = pd.to_datetime('today')
    #max_transactions_date = df['transaction_dates'].max()

    df_last_9_months = df[df['transaction_dates'] >= max_transactions_date + relativedelta(months=-12)]
    columns_to_convert = ['credit_amt', 'debit_amt','balance']

    for column in columns_to_convert:
        df_last_9_months[column] = pd.to_numeric(df_last_9_months[column], errors='coerce')
        df_last_9_months[column].fillna(0,inplace=True)


   
    credits_df_last_9_months = df_last_9_months[
        (df_last_9_months['credit_amt'] > 0) & (df_last_9_months['status'] == 'Approved')]
    
    credits_df_last_9_months_negative_balances = credits_df_last_9_months[credits_df_last_9_months['balance'] < 0]
    
    credits_negative_balances_summaries_last_9_months = credits_df_last_9_months_negative_balances.groupby('terminal').agg(
        credits_lowest_negative_balance_last_9_months=pd.NamedAgg('balance', min),
        credits_highest_negative_balance_last_9_months=pd.NamedAgg('balance', max),
        credits_earliest_negative_balance_date_last_9_months=pd.NamedAgg('transaction_dates', min),
        credits_latest_negative_balance_date_last_9_months=pd.NamedAgg('transaction_dates', max),
        credits_unique_negative_balance_dates_last_9_months=pd.NamedAgg('transaction_dates', 'nunique')).reset_index()
    
    credits_df_last_9_months_without_negative_balances = credits_df_last_9_months[credits_df_last_9_months['balance'] >= 0]
    
    credits_agent_summaries_last_9_months = credits_df_last_9_months_without_negative_balances.groupby('terminal').agg(
        total_credit_amount_last_9_months=pd.NamedAgg('credit_amt', sum),
        credits_average_transaction_size_last_9_months=pd.NamedAgg('credit_amt', 'mean'),
        credits_total_transactions_last_9_months=pd.NamedAgg('request_ref', 'nunique'),
        credits_earliest_transaction_date_last_9_months=pd.NamedAgg('transaction_dates', min),
        credits_latest_transaction_date_last_9_months=pd.NamedAgg('transaction_dates', max),
        credits_no_of_services_offered_last_9_months=pd.NamedAgg('biller', 'nunique'),
        credits_min_balance_last_9_months=pd.NamedAgg('balance', min),
        credits_max_balance_last_9_months=pd.NamedAgg('balance', max),
        credits_average_balance_last_9_months=pd.NamedAgg('balance', 'mean'),
        min_credit_amt_last_9_months=pd.NamedAgg('credit_amt', min),
        max_credit_amt_last_9_months=pd.NamedAgg('credit_amt', max),
        credits_unique_transaction_days_last_9_months=pd.NamedAgg('transaction_dates', 'nunique'),
        credits_unique_transaction_months_last_9_months=pd.NamedAgg('year_month_transaction_dates', 'nunique')).reset_index()

    credits_agent_summaries_last_9_months['credits_expected_transaction_days_last_9_months'] = (
            (max_transactions_date - (max_transactions_date + relativedelta(months=-9))) / np.timedelta64(1,
                                                                                                          'D') + 1)
    credits_agent_summaries_last_9_months['credits_daily_trading_consistency_last_9_months'] = round(
    credits_agent_summaries_last_9_months['credits_unique_transaction_days_last_9_months'] / credits_agent_summaries_last_9_months[
        'credits_expected_transaction_days_last_9_months'], 2)

    credits_agent_summaries_last_9_months['credits_average_daily_transactions_last_9_months'] = round(
    credits_agent_summaries_last_9_months['credits_total_transactions_last_9_months'] / credits_agent_summaries_last_9_months['credits_unique_transaction_days_last_9_months'])

    credits_agent_summaries_last_9_months['average_daily_credit_amt_last_9_months'] = round(
    credits_agent_summaries_last_9_months['total_credit_amount_last_9_months'] / credits_agent_summaries_last_9_months[
        'credits_expected_transaction_days_last_9_months'])

    credits_agent_summaries_last_9_months['credits_days_since_last_transaction_last_9_months'] = (
            (max_transactions_date - credits_agent_summaries_last_9_months['credits_latest_transaction_date_last_9_months']) / np.timedelta64(1,
                                                                                                                'D')).astype(
    int)

    credits_agent_summaries_last_9_months = pd.merge(credits_agent_summaries_last_9_months, credits_negative_balances_summaries_last_9_months,
                                          on='terminal', how='left')


    





    credits_agent_summaries_last_9_months['credits_lowest_negative_balance_last_9_months'].fillna(0, inplace=True)

    credits_agent_summaries_last_9_months['credits_highest_negative_balance_last_9_months'].fillna(0, inplace=True)

    credits_agent_summaries_last_9_months['credits_unique_negative_balance_dates_last_9_months'].fillna(0, inplace=True)

    credits_agent_summaries_last_9_months['credits_earliest_negative_balance_date_last_9_months'].fillna(credits_agent_summaries_last_9_months['credits_latest_transaction_date_last_9_months'], inplace=True)

    credits_agent_summaries_last_9_months['credits_latest_negative_balance_date_last_9_months'].fillna(credits_agent_summaries_last_9_months['credits_latest_transaction_date_last_9_months'], inplace=True)
    
    credits_agent_summaries_last_9_months['credits_days_since_latest_negative_balance_last_9_months'] = ((credits_agent_summaries_last_9_months[
                                                                                'credits_latest_transaction_date_last_9_months'] -
                                                                            credits_agent_summaries_last_9_months[
                                                                                'credits_latest_negative_balance_date_last_9_months']) / np.timedelta64(
        1, 'D')).astype(int)
    

    
    return credits_agent_summaries_last_9_months
#     return credits_agent_summaries_last_6_months

In [53]:
# credit_summaries_last_9_months= calculate_9_months_credit_scoring_summaries()
# credit_summaries_last_9_months

## Merges

#### Commission Summaries

In [54]:
def commissions_summaries(data):
    data=data
#     raw_data=process_dates()
#     commissions_summarries = calculate_commissions_summaries_last_12_months()
    
    last_1_months_commissions_summaries=calculate_commissions_summaries_last_1_months(data=data)
    last_3_months_commissions_summaries=calculate_commissions_summaries_last_3_months(data=data)
    last_6_months_commissions_summaries=calculate_commissions_summaries_last_6_months(data=data)
    last_9_months_commissions_summaries=calculate_commissions_summaries_last_9_months(data=data)
    last_12_months_commissions_summaries=calculate_commissions_summaries_last_12_months(data=data)
    
    commissions_summarries=pd.merge(last_1_months_commissions_summaries,last_3_months_commissions_summaries,on="terminal",how="outer")
    commissions_summarries=pd.merge(commissions_summarries,last_6_months_commissions_summaries,on="terminal",how="outer")
    commissions_summarries=pd.merge(commissions_summarries,last_9_months_commissions_summaries,on="terminal",how="outer")
    commissions_summarries=pd.merge(commissions_summarries,last_12_months_commissions_summaries,on="terminal",how="outer")
    
    return commissions_summarries
    

In [55]:
# commissions_summarries = commissions_summaries()
# commissions_summarries


### Debit Summarries

In [56]:
def debit_summaries(data):
#     raw_data=process_dates()
#     commissions_summarries = calculate_commissions_summaries_last_12_months()
    data=data
    
    debits_agent_summarries_last_1_monthss = calculate_1_months_debits_scoring_summaries(data=data)
    debits_agent_summarries_last_3_monthss = calculate_3_months_debits_scoring_summaries(data=data)
    debits_agent_summarries_last_6_monthss = calculate_6_months_debits_scoring_summaries(data=data)
    debits_agent_summarries_last_9_monthss = calculate_9_months_debits_scoring_summaries(data=data)
    debits_agent_summarries_last_12_monthss = calculate_12_months_debits_scoring_summaries(data=data)
    
    debit_summaries=pd.merge(debits_agent_summarries_last_1_monthss,debits_agent_summarries_last_3_monthss,on="terminal",how="outer")
    debit_summaries=pd.merge(debit_summaries,debits_agent_summarries_last_6_monthss,on="terminal",how="outer")
    debit_summaries=pd.merge(debit_summaries,debits_agent_summarries_last_9_monthss,on="terminal",how="outer")
    debit_summaries=pd.merge(debit_summaries,debits_agent_summarries_last_12_monthss,on="terminal",how="outer")
    
    
    
    return debit_summaries
    

In [57]:
# debit_summarie=debit_summaries()
# debit_summarie

### Credit Summarries

In [58]:
def credit_summaries(data):
    data=data
#     raw_data=process_dates()
#     commissions_summarries = calculate_commissions_summaries_last_12_months()
    
    credit_summaries_last_1_months= calculate_1_months_credit_scoring_summaries(data=data)
    credit_summaries_last_3_months= calculate_3_months_credit_scoring_summaries(data=data)
    credit_summaries_last_6_months= calculate_6_months_credit_scoring_summaries(data=data)
    credit_summaries_last_9_months= calculate_9_months_credit_scoring_summaries(data=data)
    credit_summaries_last_12_months= calculate_12_months_credit_scoring_summaries(data=data)
    
    credit_summaries=pd.merge(credit_summaries_last_1_months,credit_summaries_last_3_months,on="terminal",how="outer")
    credit_summaries=pd.merge(credit_summaries,credit_summaries_last_6_months,on="terminal",how="outer")
    credit_summaries=pd.merge(credit_summaries,credit_summaries_last_9_months,on="terminal",how="outer")
    credit_summaries=pd.merge(credit_summaries,credit_summaries_last_12_months,on="terminal",how="outer")
    
    
    
    # Ratios
    
    
    

    
    
    return credit_summaries
    

In [59]:
# credits_summaries=credit_summaries()
# credits_summaries

In [63]:
def whole_summary():
    data=raw_data=process_dates()
    commission_summarries = commissions_summaries(data=data)
    credits_summaries=credit_summaries(data=data)
    debits_summaries=debit_summaries(data=data)
    
    summaries=pd.merge(commission_summarries,debits_summaries,on="terminal", how="outer")
    
    summaries=pd.merge(summaries,credits_summaries,on="terminal", how="outer")
    
    order=['terminal'] + [col for col in summaries.columns if col != 'terminal']
    
    summaries=summaries[order]
    
    # Fillna
    
    for col in summaries.columns:
        if "date" not in col.lower() and summaries[col].isna().any():
            summaries[col].fillna(0, inplace=True)
            
    # Ratios
    
    
    summaries["ratio_of_total_commissions_last_1/3_months"]=summaries['total_commissions_amount_last_1_month']/summaries['total_commissions_amount_last_3_month']
    summaries["ratio_of_total_commissions_last_3/6_months"]=summaries['total_commissions_amount_last_3_month']/summaries['total_commissions_amount_last_6_month']
    summaries["ratio_of_total_commissions_last_6/9_months"]=summaries['total_commissions_amount_last_6_month']/summaries['total_commissions_amount_last_9_month']
    summaries["ratio_of_total_commissions_last_9/12_months"]=summaries['total_commissions_amount_last_9_month']/summaries['total_commissions_amount_last_12_month']

    summaries["ratio_of_unique_number_of_commissions_last_1/3_month"]=summaries['unique_number_of_commissions_last_1_month']/summaries['unique_number_of_commissions_last_3_month']
    summaries["ratio_of_unique_number_of_commissions_last_3/6_month"]=summaries['unique_number_of_commissions_last_3_month']/summaries['unique_number_of_commissions_last_6_month']
    summaries["ratio_of_unique_number_of_commissions_last_6/9_month"]=summaries['unique_number_of_commissions_last_6_month']/summaries['unique_number_of_commissions_last_9_month']
    summaries["ratio_of_unique_number_of_commissions_last_9/12_month"]=summaries['unique_number_of_commissions_last_9_month']/summaries['unique_number_of_commissions_last_12_month']
    
    
    summaries["ratio_of_total_credit_amount_last_1/3_months"]=summaries['total_credit_amount_last_1_months']/summaries['total_credit_amount_last_3_months']
    summaries["ratio_of_total_credit_amount_last_3/6_months"]=summaries['total_credit_amount_last_3_months']/summaries['total_credit_amount_last_6_months']
    summaries["ratio_of_total_credit_amount_last_6/9_months"]=summaries['total_credit_amount_last_6_months']/summaries['total_credit_amount_last_9_months']
    summaries["ratio_of_total_credit_amount_last_9/12_months"]=summaries['total_credit_amount_last_9_months']/summaries['total_credit_amount_last_12_months']
    
    summaries["ratio_of_min/max_credits_amt_last_1_months"]=summaries['credits_min_balance_last_1_months']/summaries['max_credit_amt_last_1_months']
#     summaries["ratio_of_min/max_credits_amt_last_2_months"]=summaries['credits_min_balance_last_2_months']/summaries['max_credit_amt_last_2_months']
    summaries["ratio_of_min/max_credits_amt_last_3_months"]=summaries['credits_min_balance_last_3_months']/summaries['max_credit_amt_last_3_months']
    summaries["ratio_of_min/max_credits_amt_last_6_months"]=summaries['credits_min_balance_last_6_months']/summaries['max_credit_amt_last_6_months']
    summaries["ratio_of_min/max_credits_amt_last_9_months"]=summaries['credits_min_balance_last_9_months']/summaries['max_credit_amt_last_9_months']
    summaries["ratio_of_min/max_credits_amt_last_12_months"]=summaries['credits_min_balance_last_12_months']/summaries['max_credit_amt_last_12_months']
    
    
    summaries['ratio_of_debits_total_debit_amount_last_1/3_months']=summaries['debits_total_debit_amount_last_1_months']/summaries['debits_total_debit_amount_last_3_months']
    summaries['ratio_of_debits_total_debit_amount_last_3/6_months']=summaries['debits_total_debit_amount_last_3_months']/summaries['debits_total_debit_amount_last_6_months']
    summaries['ratio_of_debits_total_debit_amount_last_6/9_months']=summaries['debits_total_debit_amount_last_6_months']/summaries['debits_total_debit_amount_last_9_months']
    summaries['ratio_of_debits_total_debit_amount_last_9/12_months']=summaries['debits_total_debit_amount_last_9_months']/summaries['debits_total_debit_amount_last_12_months']
    
    summaries['ratio_of_debits_min/max_balance_last_1_months']=summaries['debits_min_balance_last_1_months']/summaries['debits_max_balance_last_1_months']
    summaries['ratio_of_debits_min/max_balance_last_3_months']=summaries['debits_min_balance_last_3_months']/summaries['debits_max_balance_last_3_months']
    summaries['ratio_of_debits_min/max_balance_last_6_months']=summaries['debits_min_balance_last_6_months']/summaries['debits_max_balance_last_6_months']
    summaries['ratio_of_debits_min/max_balance_last_9_months']=summaries['debits_min_balance_last_9_months']/summaries['debits_max_balance_last_9_months']
    summaries['ratio_of_debits_min/max_balance_last_12_months']=summaries['debits_min_balance_last_12_months']/summaries['debits_max_balance_last_12_months']
    
    
    #Standard Deviations
    
    for col in summaries.columns:
        if "total" in col.lower() or "average" in col.lower() or "balance" in col.lower():
        
            std_col_name = col + "_std"  
            summaries[std_col_name] = summaries[col].std()  
            
    # Fillna for std cols      
    for col in summaries.columns:
        if "_std" in col.lower():
            # Check if there are null values and fill them with 1e-6
            if summaries[col].isnull().any():
                summaries[col] = summaries[col].fillna(1e-6)
        else: 
#             col.fillna(0)
             summaries[col] = summaries[col].fillna(0)

    with open('ISW_result_to_use.csv', 'a', newline='') as file:
                summaries.to_csv(file, header=True, index=False)
        
    return summaries

whole_summary=whole_summary()
whole_summary.head(1)



,terminal,total_commissions_amount_last_1_month,unique_number_of_commissions_last_1_month,unique_number_of_services_offered_last_1_month,number_of_months_received_commissions_last_1_month,average_commisison_last_1_months,total_commissions_amount_last_3_month,unique_number_of_commissions_last_3_month,unique_number_of_services_offered_last_3_month,number_of_months_received_commissions_last_3_month,...,ratio_of_total_credit_amount_last_9/12_months_std,ratio_of_debits_total_debit_amount_last_1/3_months_std,ratio_of_debits_total_debit_amount_last_3/6_months_std,ratio_of_debits_total_debit_amount_last_6/9_months_std,ratio_of_debits_total_debit_amount_last_9/12_months_std,ratio_of_debits_min/max_balance_last_1_months_std,ratio_of_debits_min/max_balance_last_3_months_std,ratio_of_debits_min/max_balance_last_6_months_std,ratio_of_debits_min/max_balance_last_9_months_std,ratio_of_debits_min/max_balance_last_12_months_std
0,3IS00014,0.0,0.0,0.0,0.0,0.0,599.0,2.0,2.0,2.0,...,0.0,0.0386,0.129563,0.231722,0.169472,0.000001,0.005407,0.142455,0.0,0.0


In [6]:
C_data= pd.read_csv("ISW_result_to_use.csv")
loans_data= pd.read_csv("ISW_refined_loans_data.csv")
loans_data.drop(loans_data.columns[loans_data.columns.str.contains('Unnamed')].tolist(), axis=1, inplace=True)
loans_data.rename(columns={"external_id":"terminal"},inplace=True)
loans_data.head(1)
A=pd.merge(C_data,loans_data,on="terminal",how="inner")
A.to_csv("ISW_data_for_Feature_analysis.csv")


In [7]:
FI=A.isnull().sum()[A.isnull().sum() > 1].index
FI

Index([], dtype='object')

In [ ]:
A.iloc[1197]

# Feature Selection